# Lab Exercise 1. Scraping Static Websites


This is the warmup task for the first laboratory exercise. It consists of scraping static Websites with BeautifulSoap.

 It should be completed at home and presented at the laboratory.

**Total points: 2**

### Task Description

Scrape the information about the products on the following page:
https://clevershop.mk/product-category/mobilni-laptopi-i-tableti/

For each product scrape:


*   Product title (selector `'.wd-entities-title'`)
*   Product regular price (selector `'.woocommerce-Price-amount'`)
*   Product discount price (if available), same selector as regular price
*   URL to the product page
*   Add to cart button URL

***Help: There are multiple product pages, for each page you need to send a separate request***


Save the results as a DataFrame object

You can add as many code cells as you need.

________________________________________________________________

### Requirements

Import libraries and modules that you are going to use

In [2]:
from bs4 import BeautifulSoup
import requests
import pandas as pd

### Send HTTP request to the target Website

In [3]:
response = requests.get("https://clevershop.mk/product-category/mobilni-laptopi-i-tableti/")

check the response status code

In [6]:
print(response.status_code)

200


### Parse the response content with BeautifulSoap

In [8]:
soup = BeautifulSoup(response.content, "html.parser")

### Extract data from the BeautifulSoap object using any selectors, attribute identifiers, etc.

* Product title (selector '.wd-entities-title')
* Product regular price (selector '.woocommerce-Price-amount')
* Product discount price (if available), same selector as regular price
* URL to the product page
* Add to cart button URL

In [9]:
products = soup.select('.product') 
for product in products:
    title = product.select_one('.wd-entities-title').text.strip()
    
    prices = product.select('.woocommerce-Price-amount')
    regular_price = prices[0].text.strip() if prices else None
    discount_price = prices[1].text.strip() if len(prices) > 1 else None

    product_url = product.select_one('a')['href']
    
    add_to_cart_url = product.select_one('.add_to_cart_button')['href'] if product.select_one('.add_to_cart_button') else None

    print("Title:", title)
    print("Regular Price:", regular_price)
    print("Discount Price:", discount_price)
    print("Product URL:", product_url)
    print("Add to Cart URL:", add_to_cart_url)

Title: Acer A315-23-A7KD
Regular Price: 17.590 ден
Discount Price: None
Product URL: https://clevershop.mk/product/acer-a315-23-a7kd/
Add to Cart URL: ?add-to-cart=21494
Title: Acer A315-23-R5P2
Regular Price: 27.490 ден
Discount Price: None
Product URL: https://clevershop.mk/product/acer-a315-23-r5p2/
Add to Cart URL: ?add-to-cart=21510
Title: ACER Aspire 1 A115-22
Regular Price: 18.999 ден
Discount Price: 15.999 ден
Product URL: https://clevershop.mk/product/acer-aspire-1-nx-a7pex-001/
Add to Cart URL: ?add-to-cart=20826
Title: Acer Aspire 3 A315-23-R26A
Regular Price: 29.990 ден
Discount Price: None
Product URL: https://clevershop.mk/product/acer-aspire-3-a315-23-r26a/
Add to Cart URL: ?add-to-cart=21516
Title: Acer Aspire 3 A315-58-33WK
Regular Price: 24.490 ден
Discount Price: None
Product URL: https://clevershop.mk/product/21498/
Add to Cart URL: ?add-to-cart=21498
Title: Acer Aspire 3 A315-58-33WK
Regular Price: 25.990 ден
Discount Price: None
Product URL: https://clevershop.mk/

Repeat the extraction process for each page of products

In [10]:
base_url = "https://clevershop.mk/product-category/mobilni-laptopi-i-tableti/"

all_products = []

page_number = 1
while True:
    url = f"{base_url}page/{page_number}/" if page_number > 1 else base_url
    print(f"Scraping: {url}")

    response = requests.get(url)
    print("Response Status Code:", response.status_code)

    if response.status_code != 200:
        break

    soup = BeautifulSoup(response.content, "html.parser")

    products = soup.select('.product')
    
    if not products:
        print("No more products found. Exiting.")
        break

    for product in products:
        title = product.select_one('.wd-entities-title').text.strip()
        
        prices = product.select('.woocommerce-Price-amount')
        regular_price = prices[0].text.strip() if prices else None
        discount_price = prices[1].text.strip() if len(prices) > 1 else None

        product_url = product.select_one('a')['href']
        
        add_to_cart_url = product.select_one('.add_to_cart_button')['href'] if product.select_one('.add_to_cart_button') else None

        all_products.append({
            "Title": title,
            "Regular Price": regular_price,
            "Discount Price": discount_price,
            "Product URL": product_url,
            "Add to Cart URL": add_to_cart_url
        })

    page_number += 1

df = pd.DataFrame(all_products)

print(df)

Scraping: https://clevershop.mk/product-category/mobilni-laptopi-i-tableti/
Response Status Code: 200
Scraping: https://clevershop.mk/product-category/mobilni-laptopi-i-tableti/page/2/
Response Status Code: 200
Scraping: https://clevershop.mk/product-category/mobilni-laptopi-i-tableti/page/3/
Response Status Code: 200
Scraping: https://clevershop.mk/product-category/mobilni-laptopi-i-tableti/page/4/
Response Status Code: 200
Scraping: https://clevershop.mk/product-category/mobilni-laptopi-i-tableti/page/5/
Response Status Code: 200
Scraping: https://clevershop.mk/product-category/mobilni-laptopi-i-tableti/page/6/
Response Status Code: 200
Scraping: https://clevershop.mk/product-category/mobilni-laptopi-i-tableti/page/7/
Response Status Code: 200
Scraping: https://clevershop.mk/product-category/mobilni-laptopi-i-tableti/page/8/
Response Status Code: 200
Scraping: https://clevershop.mk/product-category/mobilni-laptopi-i-tableti/page/9/
Response Status Code: 200
Scraping: https://cleversh

### Create a pandas DataFrame with the scraped products

In [12]:
df = pd.DataFrame(all_products)

Save the dataframe as `.csv`

In [13]:
df.to_csv("scraped_products.csv", index=False)